## Iterator Protocol<br>

Iter is an object. <br>
__iter__ and next ( special methods or dunder methods) are the process of the iteration over items in a container.<br>


In [1]:
seq = [ 'foo','bar' ]
for x in seq:
    print(x)

foo
bar


In [3]:
iterator = iter(seq)

while True:
    try:
        x = iterator.__next__()  # next in 2.7
        print(x)
    except StopIteration as e:
        break

foo
bar


## Generator

- The Python documentation defines a generator as follows:<br>
    A function which returns an iterator. It looks like a normal
function except that it contains yield statements
for producing a series [of] values usable in a for-loop
or that can be retrieved one at a time with the next()
function. Each yield temporarily suspends processing,
remembering the location execution state (including local
variables and pending try-statements). When the
generator resumes, it picks-up where it left-off (in contrast
to functions which start fresh on every invocation).<br>

- Generator vs function.<br>
- 1. It has `yield` <br>


In [5]:
# normal fun return None by default
def normal_fun():
    y = 2 + 2 # no default 
    
print( normal_fun () )

None


In [7]:
# 2.  generator are not executed when they are invoked, only when they are iterated over.

def simple_generator():
    print("generate")
    yield 1
    yield 2
    

simple_generator()

<generator object simple_generator at 0x000001A3BC698930>

In [9]:
# 2. Normal functions return None by default.
def normal_fun():
    y = 2 + 2
    
print( normal_fun() )

None


In [10]:
# 3. generators can be iterated over

for x in simple_generator():
    print(x)

generate
1
2


In [11]:
# 4. generators freeze their state after a yield statement.

def counter_gen(size):
    cur = 1
    while cur <= size:
        yield cur
        cur = cur + 1
        

for num in counter_gen(2):
    print(num)

1
2


In [14]:
# 5. generator return generator object

gen = counter_gen(3)
print( gen )
dir(gen)

<generator object counter_gen at 0x000001A3BC6AD048>


['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [16]:
# 6 Generator can be infinite;

def gen_forever():
    i = 1
    while True:
        yield i
        i = i + 1
        

for num in gen_forever():
    if num > 3:
        break
    print(num)

1
2
3


In [19]:
# 7 "return" stops generation

def gen_with_return():
    yield 1
    yield 2
    return  # stops here
    yield 3
    
for num in gen_with_return():
    print(num)

1
2


## Object Generators<br>
Not only functions generate, but methods also.<br>

Two ways to use:
1. returning a generator from the \__iter__method.<br>
2. object method generators.

In [21]:

class Counter3:
    
    def __init__(self, size):
        self.size = size
        
    def __iter__(self):
        cur = 1
        while cur <= self.size:
            yield cur
            cur = cur + 1
            
            
for x in Counter3(3):
    print(x)

1
2
3


In [24]:

class Counter:
    
    def __init__(self, size):
        self.size = size
        
    def count(self):
        cur = 1
        while cur <= self.size:
            yield cur
            cur = cur + 1
            
# main

c = Counter(2)
c_gen = c.count()

iter(c_gen) == c_gen # gen methods perform similarly to gen functions

True

## when to use ?<br>
a gen can replace any function that returns a list.<br>
system admin or data manipulation with huge file.

In [29]:
# 1. generator exhaust: There is NO reuse;

five = counter_gen(5)

# gen 
print([ x for x in five ] )

print([ x for x in five ]) # returning Nothing

# func
print()

print([ x for x in range(3) ])
print([ x for x in range(3) ])

[1, 2, 3, 4, 5]
[]

[0, 1, 2]
[0, 1, 2]


In [1]:
# 2. Chaining generators: Acts as a filter on sequences or manipulations of sequences.

def positive(seq):
    for x in seq:
        if x >= 0:
            yield x
            
def every_other(gen):
    for x in gen:
        try:
            yield x
            gen.__next__()
        except:
            pass
        
def double(seq):
    for x in seq:
        yield x
        yield x
        
        
# main

seq = range(-5, 5)

pos = positive(seq)
skip = every_other(pos)
two = double(skip)

print([ x for x in two ])



[0, 0, 2, 2, 4, 4]


## How to debug generators ?<br>

first,make it works using `list`. then, turn into generator.

In [ ]:

import pdb
pdb.set_trace()

seq = positive( range(5) )


--Return--
> <ipython-input-37-9c15dfa4dba6>(3)<module>()->None
-> pdb.set_trace()
(Pdb) s
> c:\anaconda\lib\site-packages\ipython\core\interactiveshell.py(3270)run_code()
-> sys.excepthook = old_excepthook
(Pdb) s
> c:\anaconda\lib\site-packages\ipython\core\interactiveshell.py(3286)run_code()
-> outflag = False
(Pdb) s
> c:\anaconda\lib\site-packages\ipython\core\interactiveshell.py(3287)run_code()
-> return outflag
(Pdb) s
--Return--
> c:\anaconda\lib\site-packages\ipython\core\interactiveshell.py(3287)run_code()->False
-> return outflag
(Pdb) n
> c:\anaconda\lib\site-packages\ipython\core\interactiveshell.py(3182)run_ast_nodes()
-> for i, node in enumerate(to_run_exec):
(Pdb) n
> c:\anaconda\lib\site-packages\ipython\core\interactiveshell.py(3183)run_ast_nodes()
-> mod = ast.Module([node])
(Pdb) 
> c:\anaconda\lib\site-packages\ipython\core\interactiveshell.py(3184)run_ast_nodes()
-> code = compiler(mod, cell_name, "exec")


In [3]:
# generators do not index or slice;

pos = positive( range(-5,5) )
pos[1]

TypeError: 'generator' object is not subscriptable

In [4]:
# solution: using itertools has islice
# -ve slicing is not available with gen

from itertools import islice

seq = islice( pos, 1, 3)
print(list( seq) )

[1, 2]


In [5]:
# Limitation: genertors have no length. Only method is to iterate over them.

In [10]:
# Genarators may be slower. Because of iteration protocol. it uses next at every step;

def iter_list():
        for x in [ 0,1,2,3,4,5]:
            pass
        
        
def iter_gen():
    def gen():
        yield 1
        yield 2
        yield 3
        yield 4
        yield 5
        
    for x in gen():
        pass


In [7]:
import timeit

t = timeit.Timer('iter_list()', setup ="from __main__ import iter_list" )
print( t.timeit() )


0.2526137999999776


In [9]:
t = timeit.Timer( 'iter_gen()', setup = "from __main__ import iter_gen")
print( t.timeit() )

0.622954400000026


### Generator consumes less memory. when compare to lists.

https://github.com/harshavl/python/blob/master/generators_sys.ipynb

In [14]:
# Gen are always True;


# for lists 
def odd_list( seq ):
    results = []
    for x in seq:
        if x % 2:
            results.append(x)
    return results

if odd_list([0,2,4]):
    print("Found Odd")
else:
    print("No Odd")
    
print( bool( odd_list([0,2,4])) )

No Odd
False


In [16]:
# for gen

def odd_gen(seq):
    for x in seq:
        if x % 2:
            yield x
            
if odd_gen([0,2,4]):
    print(" found odd")
else:
    print(" Not odd")
    
print( bool(odd_gen([0,2,4]))) # thsi is not expected. Becareful while using gen

 found odd
True


In [17]:
# A genartor in Collections: OrderedDict class uses a __iter__ generator;

from collections import OrderedDict

d = OrderedDict ()

for i in range(1,4):
    d[i] = i
    
for x in d:
    for y in d:
        print(x,y)

1 1
1 2
1 3
2 1
2 2
2 3
3 1
3 2
3 3


## Difference between list and Generator<br>

1. Repeated use of data: use list as Generator not supports;
2. Data fits to memory: use timit module to compare betwen lists vs generator.<br>
3. Operating on the sequence: Generatot won't support. for example, reversed( counter_gen(5) ). <br>
4. conversion between list and generator: list can convert to generaor;
